In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import tree
import time

df = pd.read_csv('data/conjunto_de_treinamento.csv', index_col=0)
df

,produto_solicitado,dia_vencimento,forma_envio_solicitacao,tipo_endereco,sexo,idade,estado_civil,qtde_dependentes,grau_instrucao,nacionalidade,...,possui_telefone_trabalho,codigo_area_telefone_trabalho,meses_no_trabalho,profissao,ocupacao,profissao_companheiro,grau_instrucao_companheiro,local_onde_reside,local_onde_trabalha,inadimplente
id_solicitante,,,,,,,,,,,,,,,,,,,,,
1,1,10,internet,1,M,43,1,5,0,1,...,N,,0,9.0,4.0,NaN,NaN,718.0,718.0,0
2,1,5,presencial,1,M,57,2,0,0,1,...,Y,107,0,9.0,2.0,0.0,0.0,628.0,628.0,0
3,1,15,correio,1,M,50,2,2,0,1,...,Y,84,0,9.0,2.0,11.0,NaN,456.0,456.0,0
4,1,5,presencial,1,M,26,1,0,0,1,...,Y,111,0,13.0,5.0,0.0,0.0,563.0,563.0,0
5,1,10,internet,1,F,91,2,0,0,1,...,N,,0,NaN,1.0,NaN,NaN,484.0,484.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39996,1,10,internet,1,M,39,1,1,0,1,...,Y,58,0,9.0,2.0,NaN,NaN,960.0,960.0,0
39997,1,10,internet,1,M,63,1,0,0,1,...,N,,0,NaN,NaN,NaN,NaN,867.0,867.0,0
39998,1,5,presencial,1,M,36,2,1,0,0,...,Y,119,0,9.0,2.0,11.0,0.0,681.0,681.0,1


In [2]:
df['inadimplente'].value_counts()

0    29595
1    10405
Name: inadimplente, dtype: int64

In [3]:
rows_with_null_data = df[df.isnull().any(axis=1)]
rows_with_null_data

,produto_solicitado,dia_vencimento,forma_envio_solicitacao,tipo_endereco,sexo,idade,estado_civil,qtde_dependentes,grau_instrucao,nacionalidade,...,possui_telefone_trabalho,codigo_area_telefone_trabalho,meses_no_trabalho,profissao,ocupacao,profissao_companheiro,grau_instrucao_companheiro,local_onde_reside,local_onde_trabalha,inadimplente
id_solicitante,,,,,,,,,,,,,,,,,,,,,
1,1,10,internet,1,M,43,1,5,0,1,...,N,,0,9.0,4.0,NaN,NaN,718.0,718.0,0
3,1,15,correio,1,M,50,2,2,0,1,...,Y,84,0,9.0,2.0,11.0,NaN,456.0,456.0,0
5,1,10,internet,1,F,91,2,0,0,1,...,N,,0,NaN,1.0,NaN,NaN,484.0,484.0,1
6,1,5,internet,1,M,19,1,0,0,1,...,Y,107,0,9.0,2.0,NaN,NaN,628.0,628.0,0
8,1,10,internet,1,F,27,3,0,0,1,...,N,,0,2.0,5.0,NaN,NaN,787.0,787.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,1,15,internet,1,M,30,2,2,0,1,...,Y,12,0,9.0,2.0,NaN,NaN,187.0,187.0,0
39996,1,10,internet,1,M,39,1,1,0,1,...,Y,58,0,9.0,2.0,NaN,NaN,960.0,960.0,0
39997,1,10,internet,1,M,63,1,0,0,1,...,N,,0,NaN,NaN,NaN,NaN,867.0,867.0,0


In [4]:
df.isna().sum()

produto_solicitado                      0
dia_vencimento                          0
forma_envio_solicitacao                 0
tipo_endereco                           0
sexo                                    0
idade                                   0
estado_civil                            0
qtde_dependentes                        0
grau_instrucao                          0
nacionalidade                           0
estado_onde_nasceu                      0
estado_onde_reside                      0
possui_telefone_residencial             0
codigo_area_telefone_residencial        0
tipo_residencia                      1092
meses_na_residencia                  3003
possui_telefone_celular                 0
possui_email                            0
renda_mensal_regular                    0
renda_extra                             0
possui_cartao_visa                      0
possui_cartao_mastercard                0
possui_cartao_diners                    0
possui_cartao_amex                

In [5]:
df = df.drop(['tipo_residencia'], axis=1)
df = df.drop(['meses_na_residencia'], axis=1)
df = df.drop(['profissao'], axis=1)
df = df.drop(['ocupacao'], axis=1)
df = df.drop(['profissao_companheiro'], axis=1)
df = df.drop(['grau_instrucao_companheiro'], axis=1)
df = df.dropna(subset=['local_onde_reside'])
df = df.dropna(subset=['local_onde_trabalha'])

In [6]:
# https://pbpython.com/categorical-encoding.html
# possibilit to improve using dummies
df.select_dtypes(include=['object']).head()


,forma_envio_solicitacao,sexo,estado_onde_nasceu,estado_onde_reside,possui_telefone_residencial,codigo_area_telefone_residencial,possui_telefone_celular,vinculo_formal_com_empresa,estado_onde_trabalha,possui_telefone_trabalho,codigo_area_telefone_trabalho
id_solicitante,,,,,,,,,,,
1,internet,M,RN,DF,Y,66,N,N,,N,
2,presencial,M,CE,CE,Y,107,N,Y,CE,Y,107
3,correio,M,AC,BA,Y,84,N,Y,BA,Y,84
4,presencial,M,PE,PE,Y,111,N,Y,,Y,111
5,internet,F,BA,BA,N,,N,N,,N,


In [7]:
for c in df.select_dtypes(include=['object']):
    df[c] = df[c].astype('category')
    df[c] = df[c].cat.codes


In [8]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

In [9]:
df_x = df.drop(['inadimplente'], axis=1)
df_y = df[['inadimplente']]
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.30)
clf = tree.DecisionTreeClassifier()
clf.fit(x_train, y_train)
print(clf.tree_.max_depth)
clf.score(x_test, y_test)

37


0.6240833333333333

In [10]:
parameters = {
    'n_estimators':(10, 50, 100, 200),
    'max_depth': (5, 10, 20, 40, 80),
    'min_samples_split': (0.5, 1, 2, 5, 10),
}
start = time.time() 
clf = GridSearchCV(RandomForestClassifier(), parameters, cv=5, refit=True, n_jobs=5)
clf.fit(df_x, df_y)
end = time.time() 
print('total execution time:', "%.2f" % (end-start))

total execution time: 247.18


In [11]:
print(clf.best_estimator_)
print(clf.best_score_)

RandomForestClassifier(max_depth=10)
0.7400685023127892


In [12]:
clf.cv_results_

{'mean_fit_time': array([ 0.11229982,  0.41568828,  0.84434114,  1.64111686,  0.03889623,
         0.06283126,  0.11170096,  0.13743186,  0.18211217,  0.80444808,
         1.53092995,  3.05827017,  0.18251243,  0.76734734,  1.49659586,
         3.16933289,  0.17951956,  0.83427911,  1.50229273,  2.90586143,
         0.10511851,  0.37260609,  0.74160433,  1.49853554,  0.03450742,
         0.04826627,  0.06781793,  0.10152807,  0.29304214,  1.50040674,
         2.75025005,  5.12763948,  0.29102235,  1.32864585,  2.66460404,
         5.38027101,  0.29702125,  1.32386127,  2.63036561,  5.65077147,
         0.10152845,  0.37978411,  0.80205474,  1.47785048,  0.03231301,
         0.05884242,  0.09055676,  0.13734417,  0.51534705,  2.49079189,
         5.02057381, 11.50478687,  0.61335902,  2.53691545,  4.78381276,
        10.09460363,  0.57486229,  2.43386564,  4.41778541,  9.42646728,
         0.10013242,  0.41090136,  0.74261918,  1.48952913,  0.03091707,
         0.04848065,  0.08719659, 

In [13]:
clf = RandomForestClassifier(max_depth=10, min_samples_split=5, n_estimators=50)
clf.fit(df_x, df_y)

df = pd.read_csv('data/conjunto_de_teste.csv', index_col=0)
df = df.drop(['tipo_residencia'], axis=1)
df = df.drop(['meses_na_residencia'], axis=1)
df = df.drop(['profissao'], axis=1)
df = df.drop(['ocupacao'], axis=1)
df = df.drop(['profissao_companheiro'], axis=1)
df = df.drop(['grau_instrucao_companheiro'], axis=1)
df = df.dropna(subset=['local_onde_reside'])
df = df.dropna(subset=['local_onde_trabalha'])
for c in df.select_dtypes(include=['object']):
    df[c] = df[c].astype('category')
    df[c] = df[c].cat.codes

df['inadimplente'] = clf.predict(df)
print(df['inadimplente'].value_counts())
df[['inadimplente']].to_csv('output/notebookRandomForest.csv')

0    9992
1       8
Name: inadimplente, dtype: int64
